# Introduction to Langevin Dynamics

In this notebook, we will walk through a simple demo for Langevin dynamics, where the goal is to sample from a distribution $p(x)$ using only its score function $\nabla_{x} \log p(x)$.
Here we assume a toy setting where $p(x)$ is known.
In most practical cases we only have access a dataset of samples $\mathcal{D} = \{x_0, x_1, \ldots, x_n\} \sim p(x)$, in which case we might use a technique called score matching to estimate the score function [1].

In [ ]:
from typing import Any, Callable, Dict, List, Optional, Tuple

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from tqdm import tqdm

Here we define the log pdf and the gradient of the log pdf (i.e., the score function). We also provide a function for plotting the target shape corresponding for this specific elliptical logpdf.

In [ ]:
def logpdf(
    x: torch.Tensor, 
    rx: float = 2.5, 
    ry: float = 2.5, 
    cx: float = 0.0, 
    cy: float = 0.0
) -> torch.Tensor:
    shifted_x = x - torch.tensor([cx, cy])
    scaled_x = shifted_x / torch.tensor([rx, ry])
    r = torch.norm(scaled_x, dim=-1)
    return -(r - 1) ** 2 / 0.033

def create_grad_func(
    logpdf: Callable[..., torch.Tensor], 
    **kwargs: Any
) -> Callable[[torch.Tensor], torch.Tensor]:
    def grad_logpdf(x: torch.Tensor) -> torch.Tensor:
        x.requires_grad_(True)
        log_prob = logpdf(x, **kwargs)
        return torch.autograd.grad(log_prob.sum(), x)[0]
    return grad_logpdf

def create_shape(
    rx: float = 2.5, 
    ry: float = 2.5, 
    cx: float = 0.0, 
    cy: float = 0.0
) -> Dict[str, Any]:
    return {
        "class": "Ellipse",
        "kwargs": {
            "width": 2 * rx,
            "height": 2 * ry,
            "xy": (cx, cy)
        }
    }

Here we define some utility functions for visualizing results.

In [ ]:
def plot_frame(
    particles: torch.Tensor, 
    step: int, 
    shape: Dict[str, Any], 
    figsize: Tuple[float, float] = (4, 4), 
    lim: Tuple[float, float] = (-3, 3)
) -> np.ndarray:
    particles_np = particles.detach().cpu().numpy()
    fig, ax = plt.subplots(figsize=figsize)
    ax.scatter(particles_np[step, :, 0], particles_np[step, :, 1], alpha=0.1, s=1, color="blue")
    ax.set_xlim(*lim)
    ax.set_ylim(*lim)
    ax.set_xlabel("x coord")
    ax.set_ylabel("y coord")
    ax.set_aspect("equal")
    ax.set_title(f"Langevin Sampler at t = {step}")

    shape_cls = shape["class"]
    shape_patch = getattr(matplotlib.patches, shape_cls)(
        edgecolor="red",
        facecolor="none",
        linewidth=2,
        **shape["kwargs"]
    )
    ax.add_patch(shape_patch)

    fig.canvas.draw()
    buf = fig.canvas.buffer_rgba()
    image = np.asarray(buf)

    plt.close()
    return image

def plot_trajectory(
    particles: torch.Tensor, 
    particle_idx: int, 
    axis_names: List[str] = ["x", "y"], 
    figsize: Tuple[float, float] = (10, 3), 
    lim: Tuple[float, float] = (-3, 3)
) -> None:
    particles_np = particles.detach().cpu().numpy()
    fig, ax = plt.subplots(1, len(axis_names), figsize=figsize)
    for axis, axis_name in enumerate(axis_names):
        trajectory = particles_np[:, particle_idx, axis]
        ax[axis].plot(trajectory)
        ax[axis].set_ylim(*lim)
        ax[axis].set_title(f"Trajectory of particle {particle_idx} along {axis_name}-axis")
        ax[axis].set_xlabel("timestep")
        ax[axis].set_ylabel(f"{axis_names[axis]} coord")

def frames_to_image(frames: List[np.ndarray]) -> Image.Image:
    w, h = frames[0].shape[1], frames[0].shape[0]
    collated = Image.new("RGB", (w * len(frames), h))
    for i, frame in enumerate(frames):
        collated.paste(Image.fromarray(frame), (i * w, 0))
    return collated


### Part (a)

**Finish the implementation of `langevin_update` and `sample_langevin`.**

Recall that the update equation at timestep $t$ with step size $\eta$ and random noise $\epsilon \sim \mathcal{N}(0,I)$ is

$x_{t+1} = x_t + \eta\nabla_x \log p(x) + \sqrt{2\eta}\epsilon$

In [ ]:
def langevin_update(
    grad_func: Callable[[torch.Tensor], torch.Tensor], 
    current_particles: torch.Tensor, 
    noise: torch.Tensor, 
    eta: float
) -> torch.Tensor:
    """
    Runs a single Langevin update step on current_particles.
    Returns a tensor of shape (num_particles, 2).
    """
    next_particles = current_particles + eta * grad_func(current_particles) + (2 * eta) ** 0.5 * noise
    return next_particles

def sample_langevin(
    grad_func: Callable[[torch.Tensor], torch.Tensor], 
    particles: torch.Tensor, 
    num_steps: int, 
    eta: float
) -> torch.Tensor:
    """
    Takes randomly initialized particles and runs them through a Langevin sampler.
    Returns a tensor of shape (num_steps, num_particles, 2).
    """
    particles_over_time = [particles]
    for _ in range(num_steps):
        noise = torch.randn(particles.shape, device=particles.device)
        particles = langevin_update(grad_func, particles, noise, eta)
        particles_over_time.append(particles)
    particles_over_time = torch.stack(particles_over_time)
    return particles_over_time

Now that you've completed your implementation, run the sampler and visualize results!

For Langevin sampling, you can control the number of particles (`num_particles`), the dimension of each particle (`num_dims`), the number of update steps (`num_steps`), and the step size (`eta`). You can also control the shape of the base logpdf, e.g. the radii of the x and y axes of the ellipse (`rx` and `ry`) and the center (`cx` and `cy`).

In [ ]:
def sample_and_viz_langevin(
    device: torch.device,
    langevin_kwargs: Dict[str, Any],
    ellipse_kwargs: Dict[str, Any],
    init_particles: Optional[torch.Tensor] = None
) -> List[np.ndarray]:
    if init_particles is None:
        # Initialize particles
        init_particles = torch.randn(
            langevin_kwargs["num_particles"],
            langevin_kwargs["num_dims"],
            device=device
        )
    # Run Langevin sampling
    data = sample_langevin(
        create_grad_func(logpdf, **ellipse_kwargs),
        init_particles,
        langevin_kwargs["num_steps"],
        langevin_kwargs["eta"]
    )
    # Plot results
    frames = []
    for t in tqdm(range(data.shape[0])):
        frames.append(plot_frame(data, t, create_shape(**ellipse_kwargs)))
    return frames

First, run sampling with the default hyperparameters.

*Note:* To simplify the runtime and plotting, throughout this problem you will only run the Langevin sampler for a few iterations. In practice, however, you would typically run the sampler for longer (e.g., several thousand iterations), to ensure the Markov chain has converged.

In [ ]:
device = "cpu"
langevin_kwargs = {
    "num_particles": 10000,
    "num_dims": 2,
    "num_steps": 10,
    "eta": torch.tensor([1e-2, 1e-2])
}
ellipse_kwargs = {
    "rx": 1.5,
    "ry": 1.5,
    "cx": 0.0,
    "cy": 0.0
}

frames = sample_and_viz_langevin(device, langevin_kwargs, ellipse_kwargs)
frames_to_image(frames)

Now let"s adjust the radius of the elliptical logpdf.

In [ ]:
device = "cpu"
langevin_kwargs = {
    "num_particles": 10000,
    "num_dims": 2,
    "num_steps": 10,
    "eta": torch.tensor([1e-2, 1e-2])
}
ellipse_kwargs = {
    "rx": 1.0,
    "ry": 2.5,
    "cx": 0.0,
    "cy": 0.0
}

frames = sample_and_viz_langevin(device, langevin_kwargs, ellipse_kwargs)
frames_to_image(frames)

### Part (b)

Let"s see if we can get a better fit to the elliptical logpdf in (c) by tuning each dimension of `eta`.

In [ ]:
eta = torch.tensor([1e-2, 1e-1])

device = "cpu"
langevin_kwargs = {
    "num_particles": 10000,
    "num_dims": 2,
    "num_steps": 10,
    "eta": eta
}
ellipse_kwargs = {
    "rx": 1.0,
    "ry": 2.5,
    "cx": 0.0,
    "cy": 0.0
}

frames = sample_and_viz_langevin(device, langevin_kwargs, ellipse_kwargs)
frames_to_image(frames)

### Part (c)

Here let"s move the center of the logpdf away from the origin.

In [ ]:
device = "cpu"
langevin_kwargs = {
    "num_particles": 10000,
    "num_dims": 2,
    "num_steps": 10,
    "eta": torch.tensor([1e-2, 1e-2])
}
ellipse_kwargs = {
    "rx": 1.5,
    "ry": 1.5,
    "cx": 1.0,
    "cy": 1.0
}

frames = sample_and_viz_langevin(device, langevin_kwargs, ellipse_kwargs)
frames_to_image(frames)

For the off-centered logpdf in (e), let"s try to get a better fit by tuning the initialization `init_particles`.

In [ ]:
init_particles = torch.randn(
    langevin_kwargs["num_particles"],
    langevin_kwargs["num_dims"],
    device=device
) * 1.5 + torch.tensor([1.0, 1.0], device=device)

device = "cpu"
langevin_kwargs = {
    "num_particles": 10000,
    "num_dims": 2,
    "num_steps": 10,
    "eta": torch.tensor([1e-2, 1e-2])
}
ellipse_kwargs = {
    "rx": 1.5,
    "ry": 1.5,
    "cx": 1.0,
    "cy": 1.0
}

frames = sample_and_viz_langevin(device, langevin_kwargs, ellipse_kwargs, init_particles=init_particles)
frames_to_image(frames)

### Conclusion

That's it! Congratulations on finishing the notebook.

**References**

[1] Yang Song and Stefano Ermon. Generative modeling by estimating gradients of the data distribution. NeurIPS 2019.

[2] Shreyas Kapur. Code Focused Guide on Score-Based Image Models. Blog Post 2023.

[3] Yang Song. Generative Modeling by Estimating Gradients of the Data Distribution. Blog Post. Blog Post 2021.